In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('../data/Train_dataset.csv')
train.head(1)

,Registration Number,Annual Turnover,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [4]:
from kp_auto_ml import pre_processing as pp

In [5]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [6]:
train.head(1)

,registration_number,annual_turnover,cuisine,city,restaurant_location,opening_day_of_restaurant,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,...,overall_restaurant_rating,live_music_rating,comedy_gigs_rating,value_deals_rating,live_sports_rating,ambience,lively,service,comfortablility,privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [19]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.city.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='endorsed_by',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.endorsed_by.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_theme',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_theme.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cuisine',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
    )

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=[],
    target_column='annual_turnover'
)

In [33]:
pp.process(train,model_config)

Dropping columns whose values are nullable greater than 0.7
Removing unusable values like " " and "?"
Filling NA values with median and mode for numeria and non-numeric columns.
Performing IQR based outliers cleaning on target. annual_turnover
Changing datatype of numeric columns.
Performing label encoding on columns provided.
Encoding using dummies.


,cuisine,city,restaurant_location,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,liquor_license_obtained,situated_in_a_multi_complex,dedicated_parking,...,hygiene_rating,food_rating,overall_restaurant_rating,live_music_rating,ambience,lively,service,comfortablility,privacy,annual_turnover
0,4,1,2,84.30,1,95.80,1,1,1,0,...,5,7,10.0,4.0,8.0,8,6,6,6,42000000
1,4,19,1,85.40,2,85.00,1,1,1,1,...,7,8,9.0,4.0,5.0,7,7,3,8,50000000
2,3,6,2,85.00,2,68.20,1,1,1,1,...,3,7,8.0,3.0,7.0,10,5,2,8,32500000
3,14,229,1,79.00,2,76.80,1,1,1,1,...,4,8,6.0,4.0,7.0,6,2,4,6,20000000
4,1,4,1,89.92,1,87.00,1,1,1,1,...,6,8,9.0,6.0,6.0,5,0,2,5,30000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,9,2,1,69.10,1,62.11,1,1,0,1,...,6,6,9.0,5.0,7.0,7,6,6,8,40500000
3394,1,30,1,91.00,1,96.30,1,1,0,1,...,7,8,9.0,4.0,4.0,9,4,0,5,32500000
3395,4,6,1,80.83,1,86.80,1,1,1,1,...,6,6,8.0,4.0,6.0,8,3,3,7,42500000
3396,7,1,1,79.40,1,86.00,1,1,1,1,...,5,10,7.0,3.0,7.0,6,3,3,6,53000000
